## Задача 5

Install packages if required, just delete "#" below and run cell

In [2]:
!pip install seaborn ipywidgets  

  Obtaining dependency information for seaborn from https://files.pythonhosted.org/packages/7b/e5/83fcd7e9db036c179e0352bfcd20f81d728197a16f883e7b90307a88e65e/seaborn-0.13.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/294.6 kB ? eta -:--:--
   ---- ----------------------------------- 30.7/294.6 kB ? eta -:--:--
   -------- ------------------------------ 61.4/294.6 kB 648.1 kB/s eta 0:00:01
   --------------- ---------------------- 122.9/294.6 kB 798.9 kB/s eta 0:00:01
   ------------------------------- -------- 235.5/294.6 kB 1.1 MB/s eta 0:00:01
   -------------------------------------- - 286.7/294.6 kB 1.0 MB/s eta 0:00:01
   ---------------------------------------- 294.6/294.6 kB 1.0 MB/s eta 0:00:00


In [3]:
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
sns.set_style('darkgrid')

import ipywidgets as widgets

$x_k=-5+k\frac{10}{n}$ $k=0..n$
\
$y_k=\frac{1}{1+x_k^2}$ $n=4..15$

In [27]:
a = -5
b = 5

# x_k = lambda k, n: -5 + k * (10 / n)
x_k = lambda k, n: (a+b)/2 + (b-a)/2 * np.cos((np.pi*(2*k+1))/(2*(n+1)))
y_k = lambda k, n: 1 / (1 + x_k(k, n)**2)
# y_k = lambda k, n: np.abs(x_k(k, n))

In [5]:
def getXY(n: int) -> (np.ndarray, np.ndarray):
    K = np.arange(0, n+1)
    return x_k(K, n), y_k(K, n)

**Lets plot!**

In [6]:
@widgets.interact(n=(4, 15))
def drawFunction(n: int = 4):
    plt.figure(figsize=(15, 7), dpi=70)
    x, y = getXY(n)
    plt.plot(x, y, 'or', label=f"$n={n}$")
    plt.legend(loc='best')
    plt.show()

interactive(children=(IntSlider(value=4, description='n', max=15, min=4), Output()), _dom_classes=('widget-int…

### Lagrange interpolation

$l_i(x)=\prod_{0\leq j \leq n, j\neq i}(x-x_j)$

$P_n(x) =\sum_{i=0}^n y(x_i) \cdot \frac{l_i(x)}{l_i(x_i)}$

In [7]:
def LagrangePoly(X: np.ndarray, Y: np.ndarray):
    
    assert X.size == Y.size
    n = X.size
    
    l = lambda x, i: np.prod([(x - X[j]) for j in range(n) if j != i])
    
    P = lambda x: np.sum([Y[i] * (l(x, i)) / (l(X[i], i)) for i in range(n)])
    
    return P

In [28]:
@widgets.interact(n=(4, 15))
def drawLagrangeInterpolation(n: int = 4):
    fg, ax = plt.subplots(1, 2, figsize=(16, 8), dpi=100)
    X, Y = getXY(n)
    P = LagrangePoly(X, Y)
    xx = np.linspace(X[0], X[-1], 1000)
    yy =  np.vectorize(P)(xx)
    OriginalY = np.vectorize(lambda x: 1 / (1 + x**2))(xx)
#     OriginalY = np.vectorize(lambda x: np.abs(x))(xx)
    #OriginalY = np.vectorize(lambda x: np.cos(x))(xx)
    ax[0].plot(X, Y, 'or')
    ax[0].plot(xx, yy, linestyle=':', label=r"$Lagrange \ P_n(x)$")
    ax[0].plot(xx, OriginalY, label=r"$\frac{1}{1+x^2}$")
    ax[1].plot(xx, np.abs(yy - OriginalY))
    ax[1].set_title(r'$Lagrange \ |P_n(x) - \frac{1}{1+x^2}|$')
    ax[0].legend(loc='best')

interactive(children=(IntSlider(value=4, description='n', max=15, min=4), Output()), _dom_classes=('widget-int…

### Newton interpolation

$P(x) = P(x_0) + \sum_{k=1}^{n} P(x_0,x_1,...,x_k)\prod_{j=0}^{k-1}(x-x_j)$

In [12]:
# Util Function
def NewtonFillTable(X: np.ndarray, Y: np.ndarray):
    
    assert X.size == Y.size
    n = X.size
    A = np.zeros((n, n + 1))
    A[:, 0] = X[:]
    A[:, 1] = Y[:]
    
    for j in range(2, n+1):
        for i in range(j-1, n):
            A[i, j] = (A[i, j-1] - A[i-1, j-1]) / (A[i, 0] - A[i-j+1, 0])
    
    p = np.zeros(n)
    for k in range(n):
        p[k] = A[k, k+1]
    
    return p

In [13]:
def NewtonPoly(t, X: np.ndarray, Y: np.ndarray):
    p = NewtonFillTable(X, Y)
    n = X.size
    res = p[n-1]
    for i in range(n-2, -1, -1):
        res = res * (t-X[i]) + p[i]
    return res

In [20]:
@widgets.interact(n=(4, 15))
def drawNewtonInterpolation(n: int = 4):
    fg, ax = plt.subplots(1, 2, figsize=(16, 8), dpi=100)
    X, Y = getXY(n)
    ax[0].plot(X, Y, 'or')
    
    xx = np.linspace(X[0], X[-1], 1000)
    yy = NewtonPoly(xx, X, Y)


    OriginalY = np.vectorize(lambda x: 1 / (1 + x**2))(xx)
    #OriginalY = np.vectorize(lambda x: np.cos(x))(xx)
#     P = LagrangePoly(X, Y)
#     ax[0].plot(xx, np.vectorize(P)(xx), linestyle='-.', label=r"$Lagrange \ P_n(x)$")
    ax[0].plot(xx, yy, linestyle=':', label=r"$Newton \ P_n(x)$")
    ax[0].plot(xx, OriginalY, label=r"$\frac{1}{1+x^2}$")
    ax[1].plot(xx, np.abs(yy - OriginalY))
    ax[1].set_title(r'$Newton \ |P_n(x) - \frac{1}{1+x^2}|$')
    ax[0].legend(loc='best')

interactive(children=(IntSlider(value=4, description='n', max=15, min=4), Output()), _dom_classes=('widget-int…